# Zadatak

Na adresi https://www.foi.unizg.hr/hr/dokumenti nalazi se baza dokumenata Fakulteta organizacije i informatike. Većina dokumenata zadana je u PDF formatu, no neki od dokumenata su čitki (vektorski format) dok su drugi skenirani (rasterski format). Potrebno je:
1) Implementirati program koji će skinuti sve PDF dokumente na lokalno računalo (ili Google Colab / Drive direktorij)
2) Kreirati bazu podataka (SQLite) koja će sadržavati jednu tablicu "dokument". Tablica treba sadržavati šifru dokumenta (autonumber), naslov dokumenta, putanju datoteke (path i filename na lokalnom računalu), URL adresu s koje je dokument skinut, datum, te tekstualni sadržaj dokumenta.
3) Posebno za zadnje polje u tablici potrebno je svaki dokument učitati i ekstrahirati tekst (primjerice putem Python modula PDFMiner - https://github.com/pdfminer/pdfminer.six). Ukoliko je dokument skeniran potrebno je koristiti odgovarajući OCR modul za ekstrakciju teksta (npr. PyTesseract https://pypi.org/project/pytesseract/).
4) Omogućiti pristup skupu podataka putem REST API-ja koji omogućuje isključivo pretraživanje podataka putem:(a) pretraživanja ključnih riječi u tekstu (npr. boolean search), (b) pretraživanje ključnih riječi u naslovu (c) pregled prema datumu (npr. dokumenti od DATUM do DATUM), (d) izlistavanje svih dokumenata.

# Prikupljanje podataka

In [1]:
!mkdir -p data/pdfs data/dummy_data

In [1]:
import os
import sys
import csv
import datetime
import time

import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import numpy as np
import pandas as pd

from sqlalchemy import (create_engine, MetaData, Table, Column, Integer, Text,
                        String, Double, DateTime, insert, select, update, delete)

import fitz
import pytesseract
from PIL import Image
from io import BytesIO
from pdfminer.high_level import extract_text

In [2]:
s = HTMLSession()

def get_data(url, timeout=3, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url, timeout=timeout)
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        except requests.exceptions.Timeout:
            print(f"Request timed out for URL: {url}. Retrying ({retries + 1}/{max_retries})...")
            retries += 1
        except requests.exceptions.RequestException as e:
            print(f"Error during request to {url}: {e}")
            break  # Break the loop on non-timeout errors

    print(f"Failed to fetch data from {url} after {max_retries} retries.")
    return None

## Downloading PDF

In [3]:
def download_pdf2(url, path, max_retries=3):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    retries = 0
    while retries < max_retries:
        try:
            response = s.get(url)
            response.raise_for_status()  # Checks for HTTP errors

            file_name = url.split('/')[-1]
            file_path = os.path.join(path, file_name)

            with open(file_path, 'wb') as f:
                f.write(response.content)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

In [4]:
# fixed version
def download_pdf(url, path, max_retries=3, timeout=10):
    '''
    Download a document from a given URL to a specified directory.
    URL format: /sites/default/files/[document_name].pdf
    '''
    retries = 0
    backoff = 1  # Initial backoff duration in seconds

    while retries < max_retries:
        try:
            # Check response headers before downloading the full content
            with s.get(url, stream=True, timeout=timeout) as r:
                r.raise_for_status()  # Checks for HTTP errors

                file_name = url.split('/')[-1]
                file_path = os.path.join(path, file_name)

                with open(file_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:  # Filter out keep-alive chunks
                            f.write(chunk)

            print(f"Success: {file_path}")
            break  # Exit loop on successful download
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries + 1} failed: {e}")
            sleep(backoff)  # Wait before retrying
            backoff *= 2  # Exponential backoff
            retries += 1

    if retries == max_retries:
        print(f"Failed to download file from {url} after {max_retries} retries.")

## Creating a CSV data entry/extracting useful fields

In [5]:
# fixed version
def get_metadata(url):
    '''
        Dohvaca meta podatke za zapis sa dane url adrese (/hr/dokument/[ime_dokumenta])
    '''
    soup = get_data(url)

    if not soup:
        return (None,) * 5

    ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta = (None,) * 5

    try:
        link_element = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)
        if link_element:
            link1 = link_element['href']

        datum_element = soup.find('div', class_='datum')
        if datum_element:
            datum_text = datum_element.text.strip()
            datum = datum_text.split('Kreirano: ')[1] if 'Kreirano: ' in datum_text else None

        fields = soup.find_all('div', class_='field-item even')
        if fields:
            ime_datoteke = fields[0].text.strip() if len(fields) > 0 else None
            vrsta_dokumenta = fields[1].text.strip() if len(fields) > 1 else None
            kategorija_dokumenta = fields[2].text.strip() if len(fields) > 2 else None

    except Exception as e:
        print("Error while extracting data:", e)

    return ime_datoteke, link1, datum, vrsta_dokumenta, kategorija_dokumenta

In [6]:
metadata = get_metadata('https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023')
metadata

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

## Handling pagination and going through document on the main pages

In [7]:
def extract_links(span_list):
    return [(span.find('a', href = True, target = False)['href'], span.text) for span in span_list]

In [8]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')
spans = soup.find_all('span', class_ = 'field-content') if soup else None

In [9]:
extract_links(spans)

[('/hr/dokument/izmjenjena-odluka-pok-2023', 'Izmjenjena odluka POK-a 2023.'),
 ('/hr/dokument/odluka-o-raspisivanju-izvanrednih-izbora-studentskog-zbora-suzgfoi',
  'Odluka o raspisivanju izvanrednih izbora Studentskog zbora SUZGFOI'),
 ('/hr/dokument/posebni-dio-financijskog-plana-2024-2026',
  'Posebni dio financijskog plana 2024-2026'),
 ('/hr/dokument/opci-dio-prijedlog-financijskog-plana-2024-2026',
  'Opći dio - prijedlog financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-posebnog-dijela-financijskog-plana-2024-2026',
  'Obrazloženje posebnog dijela financijskog plana 2024-2026'),
 ('/hr/dokument/obrazlozenje-opceg-dijela-financijskog-plana-2024-2026',
  'Obrazloženje općeg dijela financijskog plana 2024-2026'),
 ('/hr/dokument/dnevni-red-fakultetskog-vijeca-07122023',
  'Dnevni red Fakultetskog vijeća 07.12.2023.'),
 ('/hr/dokument/politika-privatnosti-foi-ja', 'Politika privatnosti FOI-ja'),
 ('/hr/dokument/popis-gospodarskih-subjekata-s-kojima-se-ne-smije-sklapati-u

### Handling pagination

In [10]:
soup = get_data('https://www.foi.unizg.hr/hr/dokumenti')

In [11]:
def get_next_page(soup):
    root_url = 'https://www.foi.unizg.hr'
    page = soup.find('ul', class_ = 'pager')
    next_btn = page.find('li', class_ = 'pager-next')
    if next_btn:
        url = root_url + str(next_btn.find('a')['href'])
        return url
    else:
        return None

In [12]:
def get_urls(url):
    urls = []

    while True:

        soup = get_data(url)

        url = get_next_page(soup)
        if not url:
            break
    
        print(f'Appending {url}')
        urls.append(url)
        
    return urls

In [13]:
urls = get_urls('https://www.foi.unizg.hr/hr/dokumenti')

Appending https://www.foi.unizg.hr/hr/dokumenti?page=1
Appending https://www.foi.unizg.hr/hr/dokumenti?page=2
Appending https://www.foi.unizg.hr/hr/dokumenti?page=3
Appending https://www.foi.unizg.hr/hr/dokumenti?page=4
Appending https://www.foi.unizg.hr/hr/dokumenti?page=5
Appending https://www.foi.unizg.hr/hr/dokumenti?page=6
Appending https://www.foi.unizg.hr/hr/dokumenti?page=7
Appending https://www.foi.unizg.hr/hr/dokumenti?page=8
Appending https://www.foi.unizg.hr/hr/dokumenti?page=9
Appending https://www.foi.unizg.hr/hr/dokumenti?page=10
Appending https://www.foi.unizg.hr/hr/dokumenti?page=11
Appending https://www.foi.unizg.hr/hr/dokumenti?page=12
Appending https://www.foi.unizg.hr/hr/dokumenti?page=13
Appending https://www.foi.unizg.hr/hr/dokumenti?page=14
Appending https://www.foi.unizg.hr/hr/dokumenti?page=15
Appending https://www.foi.unizg.hr/hr/dokumenti?page=16
Appending https://www.foi.unizg.hr/hr/dokumenti?page=17
Appending https://www.foi.unizg.hr/hr/dokumenti?page=18
A

In [14]:
urls[:5]

['https://www.foi.unizg.hr/hr/dokumenti?page=1',
 'https://www.foi.unizg.hr/hr/dokumenti?page=2',
 'https://www.foi.unizg.hr/hr/dokumenti?page=3',
 'https://www.foi.unizg.hr/hr/dokumenti?page=4',
 'https://www.foi.unizg.hr/hr/dokumenti?page=5']

## Scraping main pages

In [15]:
def scrape_page(url):
    '''
        Prolazi kroz linkove glavne stranice, ekstrahira /hr/dokument/ linkove i
        dohvaca meta podatke za svaki dokument
    '''
    root_url = 'https://www.foi.unizg.hr'
    print(f'Scraping {url}')
    
    soup = get_data(url)
    spans = soup.find_all('span', class_ = 'field-content')

    links = extract_links(spans)

    metadata = [get_metadata(root_url + link[0]) for link in links]

    df1 = pd.DataFrame(metadata, columns = ['naziv_datoteke', 'pdf_link', 'datum', 'vrsta_dokumenta', 'kategorija_dokumenta'])
    df2 = pd.DataFrame(links, columns = ['metadata_link', 'naslov_dokumenta'])

    return pd.concat([df1, df2], axis = 1)

In [23]:
df = scrape_page('https://www.foi.unizg.hr/hr/dokumenti')

for url in urls:
    new_df = scrape_page(url)
    df = pd.concat([df, new_df], axis = 0, ignore_index = True)

Scraping https://www.foi.unizg.hr/hr/dokumenti
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=1
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=2
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=3
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=4
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=5
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=6
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=7
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=8
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=9
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=10
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=11
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=12
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=13
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=14
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=15
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=16
Scraping https://www.foi.unizg.hr/hr/dokumenti?page=17
Scraping https://www.foi.un

In [31]:
# Sanity check
df.shape

(933, 7)

In [48]:
# Ciscenje dataframea
df.dropna(subset = ['pdf_link'], inplace = True)
df.drop_duplicates(subset=['pdf_link'], inplace = True)

In [49]:
df.shape

(849, 7)

In [50]:
df.head()

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...


In [51]:
df.naslov_dokumenta.nunique()

831

In [55]:
# Identifikacija duplikata
duplikati = df.duplicated(subset = ['naslov_dokumenta'], keep=False)

# Ispisivanje redova koji su duplikati
df[duplikati]

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta
254,odluka_o_sluzbenoj_komunikaciji.pdf,https://www.foi.unizg.hr/sites/default/files/o...,07.10.2020,Odluka,FOI,/hr/dokument/odluka-o-sluzbenoj-komunikaciji-1,Odluka o službenoj komunikaciji
286,Odluka o priznavanju stručne prakse,https://www.foi.unizg.hr/sites/default/files/o...,04.05.2020,Odluka o priznavanju stručne prakse,Odluka,/hr/dokument/odluka-o-priznavanju-strucne-prak...,Odluka o priznavanju stručne prakse
479,modeli_pracenja_rada_i_ocjenjivanja_studenata_...,https://www.foi.unizg.hr/sites/default/files/m...,24.07.2017,DS EP,None,/hr/dokument/modeli-pracenja-ds-ep-201718,Modeli praćenja DS EP 2017./18.
481,modeli_pracenja_rada_i_ocjenjivanja_studenata_...,https://www.foi.unizg.hr/sites/default/files/m...,24.07.2017,PDS EP,None,/hr/dokument/modeli-pracenja-ds-ep-201718-0,Modeli praćenja DS EP 2017./18.
572,20.5.2016._pravilnik_o_postupku_vrjednovanja_s...,https://www.foi.unizg.hr/sites/default/files/2...,25.05.2016,Pravilnik,"FOI, Kvaliteta",/hr/dokument/pravilnik-o-postupku-vrednovanja-...,Pravilnik o postupku vrednovanja studijskih pr...
643,Statut_FOI_2008._scan.pdf,https://www.foi.unizg.hr/sites/default/files/S...,06.05.2015,Statut,FOI,/hr/dokument/statut-fakulteta-organizacije-i-i...,Statut Fakulteta organizacije i informatike
645,strategija_eUcenje.pdf,https://www.foi.unizg.hr/sites/default/files/s...,06.05.2015,Strategija,Studenti,/hr/dokument/strategija-e-u%C4%8Denja,Strategija e-učenja
646,StrategijaFOI.pdf,https://www.foi.unizg.hr/sites/default/files/S...,06.05.2015,"Plan, Strategija",FOI,/hr/dokument/strate%C5%A1ki-plan-razvoja-foi-2...,Strateški plan razvoja FOI-a 2010.-2014. (skra...
647,pravilnik_osiguranje_kvalitete.pdf,https://www.foi.unizg.hr/sites/default/files/p...,06.05.2015,Pravilnik,FOI,/hr/dokument/pravilnik-o-sustavu-osiguravanja-...,Pravilnik o sustavu osiguravanja kvalitete na ...
651,FOI_odluka_sluzbena_komunikacija.pdf,https://www.foi.unizg.hr/sites/default/files/F...,06.05.2015,Odluka,FOI,/hr/dokument/odluka-o-slu%C5%BEbenoj-komunikaciji,Odluka o službenoj komunikaciji


In [56]:
df.naziv_datoteke.nunique()

832

In [57]:
# Identifikacija duplikata
duplikati = df.duplicated(subset = ['naziv_datoteke'], keep=False)

# Ispisivanje redova koji su duplikati
df[duplikati]

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta
23,odluka_o_raspisivanju_izvanrednih_izbora_za_sz...,https://www.foi.unizg.hr/sites/default/files/o...,19.09.2023,Odluka,Studenti,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora za St...
92,odluka_o_raspisivanju_izvanrednih_izbora_za_sz...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2022,Odluka,"Studenti, Studentski zbor",/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora za St...
210,statut_sz_foi.pdf,https://www.foi.unizg.hr/sites/default/files/s...,17.05.2021,Statut,"Pravilnici za studente, Studentski zbor",/hr/dokument/statut-studentskog-zbora-foi-0,Statut studentskog zbora FOI
348,pravilnik_o_zastiti_arhivskoga_i_dokumentarnog...,https://www.foi.unizg.hr/sites/default/files/p...,16.05.2019,Pravilnik,None,/hr/dokument/pravilnik-o-zastiti-arhivskoga-i-...,Pravilnik o zaštiti arhivskoga i dokumentirano...
353,pravilnik_o_zastiti_arhivskoga_i_dokumentarnog...,https://www.foi.unizg.hr/sites/default/files/p...,06.03.2019,Pravilnik,None,/hr/dokument/pravilnik-o-zastiti-arhivskoga-i-...,Pravilnik o zaštiti arhivskoga i dokumentarnog...
428,dnevni_red.pdf,https://www.foi.unizg.hr/sites/default/files/d...,16.05.2018,Dnevni red,Fakultetsko vijeće,/hr/dokument/prijedlog-dnevnog-reda-10-sjednic...,Prijedlog dnevnog reda 10. sjednice Fakultetsk...
432,dnevni_red.pdf,https://www.foi.unizg.hr/sites/default/files/d...,24.04.2018,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-8-sjednice-fakultetsko...,Dnevni red 8. sjednice Fakultetskog vijeća_17....
436,dnevni_red.pdf,https://www.foi.unizg.hr/sites/default/files/d...,23.03.2018,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-7-sjednice-fakultetsko...,Dnevni red 7. sjednice Fakultetskog vijeća_20....
443,dnevni_red.pdf,https://www.foi.unizg.hr/sites/default/files/d...,15.02.2018,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-6-sjednice-fakultetsko...,Dnevni red 6. sjednice Fakultetskog vijeća_20....
444,1._zakljucci.pdf,https://www.foi.unizg.hr/sites/default/files/1...,15.02.2018,Zaključak,Fakultetsko vijeće,/hr/dokument/zakljucci-3-sjednice-fakultetskog...,Zaključci 3. sjednice Fakultetskog vijeća Faku...


Najbitniji su linkovi, datoteke kao dnevni red, odluke, modeli pracenja i sl. mogu imati iste nazive, ali pdf-ovi su naravno na drugim adresama

In [53]:
df.to_csv('scraped_data.csv', index = False)

# SQLAlchemy

In [16]:
# Create an in-memory SQLite database
engine = create_engine('sqlite:///dokumenti.db')

metadata = MetaData()

# Define the "dokument" table with specified columns
dokument = Table('dokument', metadata,
                 Column('sifra_dokumenta', Integer, primary_key=True, autoincrement=True),
                 Column('naslov_dokumenta', String),
                 Column('putanja_datoteke', String),
                 Column('url_adresa', String),
                 Column('datum', DateTime, default=datetime.datetime.utcnow),
                 Column('tekstualni_sadrzaj', Text))

# Create the table
metadata.create_all(engine)

In [17]:
# Checking the created table
dokument.columns.keys()

['sifra_dokumenta',
 'naslov_dokumenta',
 'putanja_datoteke',
 'url_adresa',
 'datum',
 'tekstualni_sadrzaj']

# PDF text extraction

In [24]:
def is_scanned_pdf(file_path):
    text = extract_text(file_path)
    return len(text.strip()) == 0

def extract_text_from_scanned_pdf(file_path):
    document = fitz.open(file_path)
    text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        pix = page.get_pixmap()
        image = Image.open(BytesIO(pix.tobytes()))
        text += pytesseract.image_to_string(image, lang = 'hrv')

    document.close()
    return text

def extract_text_from_pdf(file_path):
    if is_scanned_pdf(file_path):
        return extract_text_from_scanned_pdf(file_path)
    else:
        return extract_text(file_path)

In [25]:
# Example usage
file_path = './data/dummy_data/scanned.pdf'
is_scanned_pdf(file_path)

True

In [29]:
text = extract_text_from_pdf(file_path)
text.strip()

"foj s=> =\n\nKLASA: 303.02/23.02/2\n(URBROJ: 2186-62.01.23-135\nVaraždin, 17. studenog 2023,\n\nTemeljem članka 80. st 2. toč. 1. Zakona o javnoj nabavi (Narodne novine broj 120/2016, 11412022),\nFakultet organizacije i informatike Varaždin, Pavinska 2, OB: 02024882310 objavjuje.\n\nPOPIS GOSPODARSKIH SUBJEKATA S KOJIMA NE SMIJE SKLAPATI\n(UGOVORE O JAVNOJ NABAVI\n\nU smislu čanka 76. Zakona o javnoj nabavi (Narodne nov/ne br 20/2016, 1142022) i Ode Fakueta\nomanizadje i nformatke Varaždin od dana 20.11.2018 godine, Klasa: 305.02/16-022, Uo: 21862.\n0810179, sob interesa postoj sa sjedećim gospodarskim subekima\n\nTomi 00, E: Kvatemika 15, Varaždin, OIB: 6963033942\n\nQOPANALITIKA 600, Dravska uica 15, Varaždn, OIB: 68221757855\n\n'AGROPROTEINKA 44, Strojarska cesta 11, Sesvete, OB: B0695452345.\n\n'AGROPROTEINKA - ENERGIJA 400, Strojarska cesta 11, 10360 Sesvete, 08: 0174095121.\n\nGeoMTech 00. Ivana Rangera 18, 42000 Varažin, OB: 32829890021\n\nMCS d00, Poljska 61, Strahoninec, OB: 

---

In [106]:
df['text'] = None

In [95]:
# download pdf -> extract text -> put it the text field
link = df.iloc[0].pdf_link
download_pdf(link, './data/dummy_data')

Success: ./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf


In [96]:
file_path = './data/dummy_data/' + link.split('/')[-1]
file_path

'./data/dummy_data/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf'

In [102]:
text = extract_text_from_pdf(file_path)
df.at[0, 'text'] = text
df.head()

,naziv_datoteke,pdf_link,datum,vrsta_dokumenta,kategorija_dokumenta,metadata_link,naslov_dokumenta,text
0,odluka_o_izmjeni_odluke_pok_2023_10_12.pdf,https://www.foi.unizg.hr/sites/default/files/o...,18.12.2023,Odluka,Kvaliteta,/hr/dokument/izmjenjena-odluka-pok-2023,Izmjenjena odluka POK-a 2023.,KLASA: 602-04/23-06/1 \nURBROJ: 2186-62-06-23-...
1,odluka-izvanredni_izbori_za_studentski_zbor-si...,https://www.foi.unizg.hr/sites/default/files/o...,12.12.2023,Odluka,Studentski zbor,/hr/dokument/odluka-o-raspisivanju-izvanrednih...,Odluka o raspisivanju izvanrednih izbora Stude...,None
6,fv_dnevni_red_2023-12-07_3._sjednica.pdf,https://www.foi.unizg.hr/sites/default/files/f...,07.12.2023,Dnevni red,Fakultetsko vijeće,/hr/dokument/dnevni-red-fakultetskog-vijeca-07...,Dnevni red Fakultetskog vijeća 07.12.2023.,None
7,politika-privatnosti-foi.pdf,https://www.foi.unizg.hr/sites/default/files/p...,24.11.2023,None,None,/hr/dokument/politika-privatnosti-foi-ja,Politika privatnosti FOI-ja,None
8,popis_gs_u_sukobu_interesa_2023.pdf,https://www.foi.unizg.hr/sites/default/files/p...,20.11.2023,Odluka,Javna nabava,/hr/dokument/popis-gospodarskih-subjekata-s-ko...,Popis gospodarskih subjekata s kojima se ne sm...,None


In [109]:
text.strip()

'KLASA: 602-04/23-06/1 \nURBROJ: 2186-62-06-23-76 \nVaraždin, 19. listopada 2023. \n\nNa temelju članka 53. Statuta Sveučilišta u Zagrebu Fakulteta organizacije i informatike (dalje u \ntekstu: Fakultet) i članka 8. i 9. Pravilnika o sustavu osiguravanja kvalitete Sveučilišta u Zagrebu \nFakulteta organizacije i informatike (u daljnjem tekstu: Pravilnik), Fakultetsko vijeće Fakulteta, \nna sjednici održanoj 19. listopada 2023. godine, donosi \n\nO D L U K U  \n\nI. \nProf. dr. sc. Renata Mekovec imenuje se za predsjednicu Povjerenstva za osiguravanje kvalitete \nSveučilišta  u  Zagrebu  Fakulteta  organizacije  i  informatike  (u  daljnjem  tekstu:  Povjerenstvo) \numjesto dosadašnjeg predsjednika prof. dr. sc. Zlatka Erjavca, a koji će kao član Uprave Fakulteta \ndjelovati u Povjerenstvu kao predstavnik Uprave i koordinator Povjerenstva.  \n\nProf.  dr.  sc.  Sandra  Lovrenčić  imenuje  se  za  zamjenicu  predsjednice  Povjerenstva  umjesto \ndosadašnje zamjenice predsjednika Povjeren